In [22]:
#untuk membaca file CSV dengan menggunakan pustaka pandas di Python.
import pickle
import pandas as pd
import pandas as pd
import re
import string
import nltk
import pymysql
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [24]:
def read_mysql_table(table, host='localhost', user='root', password='', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    query = f"SELECT * FROM {table}"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Convert the result to a Pandas DataFrame
    df = pd.DataFrame(result)
    
    # Assign column names based on the cursor description
    df.columns = [column[0] for column in cursor.description]
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()
    
    return df

In [25]:
table_name = 'input_review'
df = read_mysql_table(table_name)
df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [10]:
# membersihkan dan memproses teks pada kolom 'content'

def pre_process(text):
    text = text.lower()
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.strip()
    pisah = text.split()
    tokens = word_tokenize(text)

    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    text = stopword.remove(text)

    return text

df['content'] = df['review'].apply(pre_process)

print(df.head())

   id_review   nama     tanggal  review content
0         29  cipot  1993-11-09  bolehh  bolehh


In [11]:
review = df['content']
review.head()

0    bolehh
Name: content, dtype: object

In [12]:
# Specify the file path of the pickle file
file_path = 'Model/review.pkl'

# Read the pickle file
with open(file_path, 'rb') as file:
    data_train = pickle.load(file)

In [13]:
data_train

['update selalu log out dah login data hilang semua lama ga di buka tiba log out sendiri di benahi sayang banget aplikasi dah bagus sistemnya buruk banget jadi ragu mau tetep instal uninstal',
 'makin diperbarui makin sering erorr suka mode yg lama yg sekarang baru aja masukin data pertumbuhan malah gak update grafiknya di bulan yg lalu alhasil gabisa baca hasil 😩',
 'kenapa di update gak login coba login link email lama minta verifikasi hp tp terdaftar terus ganti nomor hp jg malah suruh isi data baru semua data ilang kalau beda email no hp dev diperbaikin aplikasinya sangat bermanfaat rekap jejak semoga data hilang kembali',
 'update malah ganti kata sandi pas buka menunya malah jadi biasa banget kecewa sama menu cara stimulasi anak sekarang malah jadi singkat padahal dulu detail banget kpsp jadi biasa banget ada gambarnya diperbaiki lebih bagus versi dulu',
 'habis update malah tambah g oke logout sendiri mau login otp g masuk',
 'update jadi bisa log in malah disuruh isi data dan k

In [14]:
# pembuatan vector kata
vectorizer = TfidfVectorizer()
train_vector = vectorizer.fit_transform(data_train)
review2 = ["".join(r) for r in review]

In [15]:
load_model = pickle.load(open('Model/model_svm2.pkl', 'rb'))

result = []

for test in review2:
    test_data = [str(test)]
    test_vector = vectorizer.transform(test_data).toarray()  # Convert to dense array
    pred = load_model.predict(test_vector)
    result.append(pred[0])

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
